# EXPLORED EDA

In [ ]:
# -------------------------------
# IMPORTS
# -------------------------------

# Main libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Checking correlation between attributes
from scipy.stats import chi2_contingency

# Scaling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Selection
from sklearn.feature_selection import SelectKBest, f_classif

# Split
from sklearn.model_selection import train_test_split

# Optimize models
from sklearn.model_selection import GridSearchCV

# Plot models
from sklearn.tree import plot_tree

# Save models
from pickle import dump

# CLASSIFICATION MODELS
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# REGRESSION MODELS
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score

In [ ]:
# -------------------------------
# STEP 0) LOAD RAW DATAFRAME
# -------------------------------
print("STEP 0) LOAD RAW DATAFRAME")

# ||||||||||||||||||
# ||||| INPUTS |||||
# ||||||||||||||||||
data_separator = ","
input_path = "../data/raw/internal-link.csv"

# Read DataFrame
df_raw=pd.read_csv(input_path, sep = data_separator)

print("- ✅ DataFrame loaded sucessfully!")

In [ ]:
# -------------------------------
# STEP 1) EXPLORE DATAFRAME
# -------------------------------
print("STEP 1) EXPLORE DATAFRAME")

# Copy  of previous DataFrame
df_S1 = df_raw.copy()

# Print shape
print(f"- ℹ️ Shape of the original DataFrame: {df_S1.shape}")

# Show first rows
print("- ℹ️ Content of the original DataFrame:")
display(df_S1.head(5))

# Show dataframe info
print("- ℹ️ Info of the original DataFrame (dataType and non-null values):")
df_S1.info(verbose=True, show_counts=True)

# Ordered info (fewest non-null first)
print("\n- ℹ️ Ordered info by number of non-null values:")
ordered_info = pd.DataFrame({
    "Column": df_S1.columns,
    "Non-Null Count": df_S1.notnull().sum(),
    "Null Count": df_S1.isnull().sum(),
    "Dtype": df_S1.dtypes.astype(str)
}).sort_values(by="Non-Null Count", ascending=True)

display(ordered_info)

# Count unique attributes (unsorted)
df_S1_summary = pd.DataFrame({
    "Column": df_S1.columns,
    "Unique_Count": df_S1.nunique().values
})
print("\n - ℹ️ Final DataFrame unique attributes (unsorted):")
display(df_S1_summary)

# Ordered summary (fewest unique values first)
print("\n - ℹ️ Ordered unique attributes (fewest unique first):")
df_S1_summary_ordered = df_S1_summary.sort_values(by="Unique_Count", ascending=True)
display(df_S1_summary_ordered)

# Automatic Warning for high-uniqueness columns
unique_counts = df_S1.nunique()
high_unique_cols = unique_counts[unique_counts == len(df_S1)].index.tolist()
if len(high_unique_cols) > 0:
    print("\n - ⚠️ Consider dropping the following columns for having UNIQUE values for EVERY row:")
    for col in high_unique_cols:
        print(f"   - {col}")

CONCLUSIONS:
- all my attributes are considered relevant
- there are not non-null values in the data -> nice
- Outcome is going to be the target variable of the analysis

In [ ]:
# -------------------------------
# STEP 2) SELECT RELEVANT ATTRIBUTES
# -------------------------------
print("STEP 2) SELECT RELEVANT ATTRIBUTES")

# Copy  of previous DataFrame
df_S2 = df_S1.copy()

# ||||||||||||||||||
# ||||| INPUTS |||||
# ||||||||||||||||||
df_S2=df_S2.drop(labels=[], axis =1) # Drop non-relevant attributes

# Print results
print("- ✅ Non-Relevant attributes have been dropped.")
print(f" - ℹ️ Previous df's columns: {len(df_S1.columns)}")
print(f" - ℹ️ Current df's columns: {len(df_S2.columns)}")
print(f" - ℹ️ Final DataFrame shape: {df_S2.shape}")
display(df_S2.head())

# Count attributes
df_S2_summary = pd.DataFrame({
    "Column": df_S2.columns,
    "Unique_Count": df_S2.nunique().values
})
print(" - ℹ️ Final DataFrame unique attributes:")
display(df_S2_summary)

In [ ]:
# -------------------------------
# STEP 3) REMOVE DUPLICATES
# -------------------------------
print("STEP 3) REMOVE DUPLICATES")

# Copy  of previous DataFrame
df_S3 = df_S2.copy()

num_duplicates=df_S3.duplicated().sum()
if num_duplicates == 0:
    df_S3=df_S3
    print("- ✅ Previous DataFrame does not contain duplicates:")
    print(" - ℹ️ Previous df's shape: ",df_S2.shape)
    print(" - ℹ️ Current df's  shape: ",df_S3.shape)
    print(" - ℹ️ These are the dropped duplicates:")
else:
    df_S3_duplicates=df_S3[df_S3.duplicated()] #Works as bool mask
    df_S3=df_S3.drop_duplicates()
    print("- ⚠️ Previous DataFrame contained " + str(num_duplicates) + " duplicates that have been dropped:")
    print(" - ℹ️ Previous df's shape: ",df_S2.shape)
    print(" - ℹ️ Current df's  shape: ",df_S3.shape)
    print(" - ℹ️ These are the dropped duplicates:")
    display(df_S3_duplicates)

In [ ]:
# -------------------------------
# STEP 4) CLASSIFY ATTRIBUTES AND TARGET VARIABLE
# -------------------------------
print("STEP 4) CLASSIFY ATTRIBUTES AND TARGET VARIABLE")

# Copy  of previous DataFrame
df_S4 = df_S3.copy()

# ||||||||||||||||||
# ||||| INPUTS |||||
# ||||||||||||||||||
var_type_proposal_threshold = 0.25  # [%] Under this percentage of unique values, the attribute is proposed as CATEGORIC
float_discrete_threshold = min(30, round(0.02 * len(df_S4))) # Dynamic threshold for FLOAT to be considered DISCRETE

# List of columns
columns = df_S4.columns.tolist()

# Iterate through columns
category_var_auto = []
numeric_var_auto = []
for col in df_S4.columns:
    col_data = df_S4[col].dropna()
    total_rows = len(df_S4)

    # Skip empty columns
    if total_rows == 0:
        continue

    unique_count = col_data.nunique()
    unique_ratio = unique_count / total_rows * 100
    col_dtype = str(df_S4[col].dtype)

    # Case 1: text-based columns
    if col_dtype in ["object", "category"]:
        category_var_auto.append(col)
        continue

    # Case 2: integer columns
    if col_dtype.startswith("int"):
        if unique_ratio <= var_type_proposal_threshold:
            category_var_auto.append(col)
        else:
            numeric_var_auto.append(col)
        continue

    # Case 3: float columns
    if col_dtype.startswith("float"):
        if unique_ratio <= var_type_proposal_threshold:
            category_var_auto.append(col)
        else:
            numeric_var_auto.append(col)
        continue

# Print proposed Data Types
print("- ℹ️ Proposed CATEGORY Attributes: " + str(category_var_auto))
print("- ℹ️ Proposed NUMERIC Attributes: " + str(numeric_var_auto))

CONCLUSIONS:
- Proposal of CATEGORY vs NUMERIC values matches perfectly the types given by the exercise

In [ ]:
# ||||||||||||||||||
# ||||| INPUTS |||||
# ||||||||||||||||||
y_var = "Outcome" # Confirm target variable
if_target_is_binary_treat_as_categoric = True # Confirm treatment for target variable

In [ ]:
# Confirm categories and target variable
category_att = []
numeric_att = []
for att in category_var_auto:
    if att != y_var:
        category_att.append(att)
for att in numeric_var_auto:
    if att != y_var:
        numeric_att.append(att)

# Checking CATEGORY attributes
binary_att = []
multiclass_att = []
constant_att = []
for att in category_att:
    att_unique_values = df_S4[att].nunique()

    if att_unique_values == 2:
        binary_att.append(att)
    elif att_unique_values > 2:
        multiclass_att.append(att)
    else:
        constant_att.append(att)

# Checking NUMERIC attributes
discrete_att = []
continuos_att = []
for att in numeric_att:
    att_dtype = df_S4[att].dtype.kind
    unique_count = df_S4[att].nunique()

    if att_dtype in ['i', 'u']:
        discrete_att.append(att)
    elif att_dtype == 'f' and unique_count < float_discrete_threshold:
        discrete_att.append(att)
    else:
        continuos_att.append(att)

# Checking TARGET variable
y_unique_values = df_S4[y_var].nunique()
y_dtype = df_S4[y_var].dtype.kind

if y_var in category_var_auto:
    if y_unique_values == 2:
        y_var_type = "CATEGORIC"
        y_var_subtype = "BINARY"
    elif y_unique_values > 2:
        y_var_type = "CATEGORIC"
        y_var_subtype = "MULTICLASS"
    else:
        y_var_type = "CATEGORIC"
        y_var_subtype = "CONSTANT"
else:
    if y_unique_values == 2 and if_target_is_binary_treat_as_categoric:
        y_var_type = "CATEGORIC"
        y_var_subtype = "BINARY"
    elif y_dtype in ['i', 'u']:
        y_var_type = "NUMERIC"
        y_var_subtype = "DISCRETE"
    elif y_dtype == 'f' and y_unique_values < float_discrete_threshold:
        y_var_type = "NUMERIC"
        y_var_subtype = "DISCRETE"
    else:
        y_var_type = "NUMERIC"
        y_var_subtype = "CONTINUOUS"

# Print results
print("- ℹ️ Confirmed CATEGORY Attributes:")
print("   ↳ BINARY: " + str(binary_att))
print("   ↳ MULTICLASS: " + str(multiclass_att))
print("   ↳ CONSTANT: " + str(constant_att))
print("- ℹ️ Confirmed NUMERIC Attributes: " + str(numeric_att))
print("   ↳ DISCRETE: " + str(discrete_att))
print("   ↳ CONTINUOUS: " + str(continuos_att))
print("- ℹ️ Confirmed TARGET Variable: " + y_var + " -> " + y_var_type + " and " + y_var_subtype)

In [ ]:
# ||||||||||||||||||
# ||||| INPUTS |||||
# ||||||||||||||||||
make_plots_UNIVARIANT = True                # Draw plots?
make_plots_MULTIVARIANT_TARGET = True       # Draw plots?
make_plots_MULTIVARIANT_ATTRIBUTES = True   # Draw plots?
figHeight_unit = 8                          # Unitary figure height
figWidth_unit = 12                          # Unitary figure width
num_cols = 2                                # Number of columns per plot
my_palette = "pastel"                       # Palette
my_font_size = 15                           # Font size
font_size_titles = 25                       # Font size for titles
num_values_to_plot = 40                     # Max number of different values to plot (for CATEGORY_var)
num_bins = 100                              # Num of bins (for NUMERIC_var plots)
category_combi_att = ""                     # Combination attribute for multivariant analysis (must be a CATEGORIC attribute)
y_var_highlighting_color = "green"          # Color to highlight target variable

 # Validation
if not category_att:
    print("- ℹ️ There are no CATEGORIC attributes in the DataFrame")
elif category_combi_att in category_att:
    print("- ✅ Sucessfull verification: combination attribute " +  category_combi_att + " is CATEGORIC")
elif category_combi_att in numeric_att:
    raise ValueError("❌ Combination attribute " +  category_combi_att + " for multivariant analysis must be a CATEGORY attribute!")
else:
    raise ValueError("❌ Combination attribute " +  category_combi_att + " does not exist in the DataFrame")

In [ ]:
# -------------------------------
# STEP 5 - UNIVARIABLE ANALYSIS
# -------------------------------
print("STEP 5 - UNIVARIABLE ANALYSIS")

if not make_plots_UNIVARIANT:
    print("⚠️ UNIVARIABLE ANALYSIS is not printed, set make_plots_UNIVARIANT = True")
else:

    # Copy of previous DataFrame
    df_S5 = df_S4.copy()

    # Target highlighting styles
    target_box_style = dict(facecolor='none', edgecolor=y_var_highlighting_color, linewidth=5)
    target_title_style = dict(fontsize= font_size_titles, color=y_var_highlighting_color, fontweight='bold')

    # CATEGORY VARIABLES (including target if categorical)
    print("🏷️ CATEGORY VARIABLES")

    if not category_att and y_var_type == "NUMERIC":
        print("  ⚠️ This type of plot is non applicable for this case, because there are not CATEGORIC variables in the DataFrame")
    else:    
        var_to_plot = category_att.copy()
        if y_var_type == "CATEGORIC" and y_var not in var_to_plot:
            var_to_plot.insert(0, y_var)

        num_rows = math.ceil(len(var_to_plot) / num_cols)

        fig, axes = plt.subplots(
            nrows=num_rows,
            ncols=num_cols,
            figsize=(figWidth_unit * num_cols, figHeight_unit * num_rows)
        )

        axes = axes.flatten()
        idx = 0

        for col in var_to_plot:
            unique_count = df_S5[col].nunique()

            if unique_count > num_values_to_plot:
                order = df_S5[col].value_counts().head(num_values_to_plot).index
            else:
                order = df_S5[col].value_counts().index

            sns.countplot(
                ax=axes[idx],
                data=df_S5,
                x=col,
                hue=col,
                palette=my_palette,
                order=order,
                legend=False
            )
            axes[idx].tick_params(axis='x', rotation=90, labelsize=my_font_size)
            axes[idx].set_xlabel("")

            # Highlight target
            if col == y_var:
                axes[idx].set_title(col, **target_title_style)
                axes[idx].add_patch(
                    plt.Rectangle((0, 0), 1, 1, transform=axes[idx].transAxes, **target_box_style)
                )
            else:
                axes[idx].set_title(col, fontdict = {"fontsize": font_size_titles})

            # Add truncated info
            if unique_count > num_values_to_plot:
                msg = f"There are {unique_count} values,\nbut only {num_values_to_plot} have been plotted"
                axes[idx].text(
                    0.5, 0.9, msg,
                    transform=axes[idx].transAxes,
                    fontsize=my_font_size,
                    color="red",
                    ha="center", va="top",
                    bbox=dict(facecolor="grey", alpha=0.25, edgecolor="red")
                )

            idx += 1

        # Hide unused axes
        for j in range(idx, len(axes)):
            axes[j].set_visible(False)

        plt.tight_layout()
        plt.show()

    # NUMERIC VARIABLES (including target if numeric)
    print("🔢 NUMERIC VARIABLES")

    if not numeric_att and y_var_type == "CATEGORIC":
        print("  ⚠️ This type of plot is non applicable for this case, because there are not NUMERIC variables in the DataFrame")
    else: 
        var_to_plot = numeric_att.copy()
        if y_var_type == "NUMERIC" and y_var not in var_to_plot:
            var_to_plot.insert(0, y_var)

        num_rows = math.ceil(len(var_to_plot) / num_cols)

        fig, axes = plt.subplots(
            nrows=num_rows * 2,
            ncols=num_cols,
            figsize=(figWidth_unit * num_cols, figHeight_unit * num_rows),
            gridspec_kw={'height_ratios': [4, 0.5] * num_rows}
        )

        var_idx = 0
        for row in range(num_rows):
            for col in range(num_cols):

                if var_idx >= len(var_to_plot):
                    axes[row * 2, col].set_visible(False)
                    axes[row * 2 + 1, col].set_visible(False)
                    continue

                colname = var_to_plot[var_idx]

                # Histogram
                sns.histplot(
                    ax=axes[row * 2, col],
                    data=df_S5,
                    x=colname,
                    bins=num_bins
                )
                axes[row * 2, col].set_xlabel("")

                # Boxplot
                sns.boxplot(
                    ax=axes[row * 2 + 1, col],
                    data=df_S5,
                    x=colname
                )
                axes[row * 2 + 1, col].set_xlabel("")

                # Highlight target
                if colname == y_var:
                    axes[row * 2, col].set_title(colname, **target_title_style)
                    axes[row * 2 + 1, col].set_title(colname, **target_title_style)

                    axes[row * 2, col].add_patch(
                        plt.Rectangle((0, 0), 1, 1, transform=axes[row * 2, col].transAxes, **target_box_style)
                    )
                    axes[row * 2 + 1, col].add_patch(
                        plt.Rectangle((0, 0), 1, 1, transform=axes[row * 2 + 1, col].transAxes, **target_box_style)
                    )
                else:
                    axes[row * 2, col].set_title(colname, fontdict = {"fontsize": font_size_titles})

                var_idx += 1

        plt.tight_layout()
        plt.show()


In [ ]:
CONCLUSIONS:
- there are around double negative than positive in diabetes
- Glucose, BloodPressure, SkinThickness, Insulin, BMI present many zero values, which are impossible, so they will be taken as missing values
- most of the data present outliers

In [ ]:
# -------------------------------
# STEP 6 - MULTIVARIANT ANALYSIS - ATTRIBUTES VS TARGET
# -------------------------------
print("STEP 6 - MULTIVARIANT ANALYSIS - ATTRIBUTES VS TARGET")

if not make_plots_MULTIVARIANT_TARGET:
    print("⚠️ MULTIVARIANT ANALYSIS - ATTRIBUTES VS TARGET is not printed, set make_plots_MULTIVARIANT_TARGET = True")
else:
    # Copy  of previous DataFrame
    df_S6 = df_S4.copy()
    print("\n 🔢 NUMERIC Attributes VS 🏷️ CATEGORY Target")

    if not numeric_att:
        print("  ⚠️ This type of plot is non applicable for this case, because there are not NUMERIC attributes in the DataFrame")
    elif y_var_type == "NUMERIC":
        print("  ⚠️ This type of plot is non applicable for this case, because the target variable is NUMERIC")
    else:
        # Set plotting variables
        var_to_plot=numeric_att # Variable type for this plot
        num_rows = math.ceil(len(var_to_plot) / num_cols) # Number of rows for the figure

        # Create subplots with custom height ratios
        fig, axes = plt.subplots(
            nrows = num_rows,
            ncols = num_cols * 2,
            figsize=(figWidth_unit * num_cols, figHeight_unit * num_rows),
            gridspec_kw={'width_ratios': [3, 1] * num_cols})

        # Loop through variables
        var_idx = 0
        for row in range(num_rows):
            for col in range(num_cols):
                if var_idx >= len(var_to_plot):
                    # Hide unused subplots
                    axes[row, col * 2].set_visible(False)
                    axes[row , col* 2 + 1].set_visible(False)
                    continue
                
                sns.stripplot(
                    ax = axes[row, col * 2],
                    data = df_S6,
                    x = y_var,
                    y = var_to_plot[var_idx],
                    hue = y_var,
                    alpha = 0.3,
                    legend = False)
                axes[row, col * 2].set_ylabel(var_to_plot[var_idx],fontdict = {"fontsize": my_font_size})
                axes[row, col * 2].grid(True)

                sns.boxplot(
                    ax = axes[row, col * 2 + 1],
                    data = df_S6,
                    x = y_var,
                    y = var_to_plot[var_idx],
                    hue = y_var,
                    palette = my_palette,
                    legend = False)
                axes[row, col * 2 + 1].set_ylabel("")
                axes[row, col * 2 + 1].grid(True)
                axes[row, col * 2 + 1].set_yticklabels([])
                
                var_idx += 1

        # Adjust layout
        plt.tight_layout()
        plt.show()
        
    print("\n 🔢 NUMERIC Attributes VS 🔢 NUMERIC Target")

    if not numeric_att:
        print("  ⚠️ This type of plot is non applicable for this case, because there are not NUMERIC attributes in the DataFrame")
    elif y_var_type == "CATEGORIC":
        print("  ⚠️ This type of plot is non applicable for this case, because the target variable is CATEGORIC")
    else:
        # Set plotting variables
        var_to_plot=numeric_att # Variable type for this plot
        num_rows = math.ceil(len(var_to_plot) / num_cols) # Number of rows for the figure

        # Create subplots with custom height ratios
        fig, axes = plt.subplots(
            nrows = num_rows * 2,
            ncols = num_cols,
            figsize=(figWidth_unit * num_cols, figHeight_unit * num_rows),
            gridspec_kw={'height_ratios': [4, 1] * num_rows})

        # Loop through variables
        var_idx = 0
        for row in range(num_rows):
            for col in range(num_cols):
                if var_idx >= len(var_to_plot):
                    # Hide unused subplots
                    axes[row * 2, col].set_visible(False)
                    axes[row * 2 + 1, col].set_visible(False)
                    continue

                # Regplot (top)
                sns.regplot(
                    ax = axes[row * 2, col],
                    data = df_S6,
                    x = var_to_plot[var_idx],
                    y = y_var,
                    scatter_kws = {'s': my_font_size, 'alpha': 0.6},
                    line_kws = {'color': 'red'})

                # Boxplot (bottom)
                sns.heatmap(
                    ax = axes[row * 2 + 1, col],
                    data = df_S6[[var_to_plot[var_idx], y_var]].corr(),
                    annot = True,
                    fmt = ".2f",
                    cbar = False)

                var_idx += 1

        # Adjust layout
        plt.tight_layout()
        plt.show()
    
    print("\n 🏷️ CATEGORY Attributes VS 🔢 NUMERIC Target")

    if not category_att:
        print("  ⚠️ This type of plot is non applicable for this case, because there are not CATEGORIC attributes in the DataFrame")
    elif y_var_type == "CATEGORIC":
        print("  ⚠️ This type of plot is non applicable for this case, because the target variable is CATEGORIC")
    else:        
        # Set plotting variables
        var_to_plot=category_att # Variable type for this plot
        num_rows = math.ceil(len(var_to_plot) / num_cols) # Number of rows for the figure

        # Create subplots
        fig, axes = plt.subplots(
            nrows = num_rows,
            ncols = num_cols,
            figsize = (figWidth_unit * num_cols, figHeight_unit * num_rows))

        # flatten the axes array for easier access
        axes = axes.flatten()  

        # Loop through variables
        idx = 0
        for col in var_to_plot:

            # Count unique values
            unique_count = df_S6[col].nunique()
            
            # Limit the number of plotted categories if there are more than num_values_to_plot
            if unique_count > num_values_to_plot:
                order = df_S6[col].value_counts().head(num_values_to_plot).index
            else:
                order = df_S6[col].value_counts().index
            
            # Create barplot
            sns.barplot(
                ax=axes[idx],
                data = df_S6,
                x = col,
                y = y_var,
                hue = category_combi_att,
                order = order)
            axes[idx].tick_params(axis='x', rotation=90, labelsize=10)
            
            # Add text box if truncated
            if unique_count > num_values_to_plot:
                msg = "There are " + str(unique_count) + " different values,\nbut only " + str(num_values_to_plot) + " have been plotted"
                axes[idx].text(
                    x = 0.5,
                    y = 0.9,
                    s = msg,
                    transform=axes[idx].transAxes,
                    fontsize = my_font_size,
                    color = 'red',
                    ha = 'center',
                    va = 'top',
                    bbox = dict(facecolor='grey', alpha=0.5, edgecolor='red'))
            
            idx += 1

        # Turn off unused axes if there are any
        for j in range(idx, len(axes)):
            axes[j].set_visible(False)

        # Adjust layout and display
        plt.tight_layout()
        plt.show()

    print("\n 🏷️ CATEGORY Attributes with 🏷️ Combined CATEGORY Target")

    if not category_att:
        print("  ⚠️ This type of plot is non applicable for this case, because there are not CATEGORIC attributes in the DataFrame")
    elif y_var_type == "NUMERIC":
        print("  ⚠️ This type of plot is non applicable for this case, because the target variable is NUMERIC")
    else:
        # Set plotting variables
        hue_order = sorted(df_S6[y_var].dropna().unique().tolist()) # Determine hue order dynamically
        var_to_plot=category_att # Variable type for this plot
        num_rows = math.ceil(len(var_to_plot) / num_cols) # Number of rows for the figure

        # Create subplots
        fig, axes = plt.subplots(
            nrows = num_rows,
            ncols = num_cols,
            figsize=(figWidth_unit * num_cols, figHeight_unit * num_rows))
            
        # Flatten the axes array for easier access
        axes = axes.flatten()  

        # Loop through variables
        idx = 0
        for col in var_to_plot:
            # Count unique values
            unique_count = df_S6[col].nunique()
            
            # Limit the number of plotted categories if there are more than num_values_to_plot
            if unique_count > num_values_to_plot:
                order = df_S6[col].value_counts().head(num_values_to_plot).index
            else:
                order = df_S6[col].value_counts().index
            
            # Create countplot
            sns.countplot(
                ax = axes[idx],
                data = df_S6,
                x = col,
                hue = y_var,
                hue_order = hue_order,
                palette = my_palette,
                order = order)
            axes[idx].tick_params(axis='x', rotation=90, labelsize=my_font_size)
            
            # Add text box if truncated
            if unique_count > num_values_to_plot:
                msg = "There are " + str(unique_count) + " different values,\nbut only " + str(num_values_to_plot) + " have been plotted"
                axes[idx].text(
                    x = 0.5,
                    y = 0.9,
                    s = msg,
                    transform=axes[idx].transAxes,
                    fontsize=my_font_size,
                    color='red',
                    ha='center',
                    va='top',
                    bbox=dict(facecolor='grey', alpha=0.25, edgecolor='red'))
            
            idx += 1

        # Turn off unused axes if there are any
        for j in range(idx, len(axes)):
            axes[j].set_visible(False)

        # Adjust layout and display
        plt.tight_layout()
        plt.show()


CONCLUSIONS:
- all attributes seem to have impact into getting Outcome = 1 (positive), the higher the attribute, the higher the changes to get Outcome = 1

In [ ]:
# -------------------------------
# STEP 7 - MULTIVARIANT ANALYSIS - ATTRIBUTES VS ATTRIBUTES
# -------------------------------
print("STEP 7 - MULTIVARIANT ANALYSIS - ATTRIBUTES VS ATTRIBUTES")

if not make_plots_MULTIVARIANT_ATTRIBUTES:
    print("⚠️ MULTIVARIANT ANALYSIS - ATTRIBUTES VS ATTRIBUTES is not printed, set make_plots_MULTIVARIANT_ATTRIBUTES = True")
else:
    # Copy of previous DataFrame
    df_S7 = df_S4.copy()

    print("\n 🔢 NUMERIC Attributes VS 🔢 NUMERIC Attributes")

    var_to_plot = numeric_att
    num_rows = len(var_to_plot) - 1  # Number of rows (one less than number of variables)

    # Create subplots with two stacked plots (regplot + heatmap) per variable pair
    fig, axes = plt.subplots(
        nrows = num_rows * 2,
        ncols = len(var_to_plot) - 1,
        figsize=(figWidth_unit * (len(var_to_plot) - 1), figHeight_unit * num_rows),
        gridspec_kw={'height_ratios': [4, 1] * num_rows})

    # Flatten axes for easy handling
    axes = np.array(axes)

    # Track subplot usage
    for row in range(num_rows):
        n_cols = len(var_to_plot) - row - 1  # Decreasing number of columns each row
        for col in range(n_cols):

            # Top: regplot
            sns.regplot(
                ax = axes[row * 2, col],
                data = df_S7,
                x = var_to_plot[row + col + 1],
                y = var_to_plot[row],
                scatter_kws = {'s': my_font_size, 'alpha': 0.6},
                line_kws = {'color': 'red'})
            axes[row * 2, col].set_xlabel(var_to_plot[row + col + 1], fontsize=20)
            axes[row * 2, col].set_ylabel(var_to_plot[row], fontsize=20)

            # Show Y label only for first plot in row
            if col == 0:
                axes[row * 2, col].set_ylabel(var_to_plot[row], fontsize=my_font_size)
            else:
                axes[row * 2, col].set_ylabel("")

            # Bottom: heatmap (correlation)
            sns.heatmap(
                ax = axes[row * 2 + 1, col],
                data = df_S7[[var_to_plot[row + col + 1], var_to_plot[row]]].corr(),
                annot = True,
                fmt = ".2f",
                cbar = False,
                annot_kws = {"size": 20})

        # Hide unused subplots on the right for this row
        for col in range(n_cols, len(var_to_plot) - 1):
            axes[row * 2, col].set_visible(False)
            axes[row * 2 + 1, col].set_visible(False)

    # Adjust layout and show
    plt.tight_layout()
    plt.show()

    print("\n 🏷️🔢 ALL Attributes VS 🏷️🔢 ALL Attributes")

    # Encode categorical variables using the Series.factorize() method
    for col in category_att:
        codes, uniques = df_S7[col].factorize()
        df_S7[col] = codes  # replace column with integer codes

    # CATEGORIC ATTRIBUTES HEATMAP
    if len(category_att) > 1:
        corr_cat = df_S7[category_att].corr()
        fig = plt.figure(figsize=(figWidth_unit, figHeight_unit))
        plt.title("ONLY CATEGORIC ATTRIBUTES", fontsize=font_size_titles + 2, fontweight="bold")
        sns.heatmap(
            data=corr_cat,
            annot=True,
            vmin=-1,
            vmax=1,
            fmt=".2f",
            annot_kws={"size": my_font_size}
        )
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ Not enough categorical attributes for a correlation matrix.")

    # NUMERIC ATTRIBUTES HEATMAP
    if len(numeric_att) > 1:
        corr_num = df_S7[numeric_att].corr()
        fig = plt.figure(figsize=(figWidth_unit, figHeight_unit))
        plt.title("ONLY NUMERIC ATTRIBUTES", fontsize=font_size_titles + 2, fontweight="bold")
        sns.heatmap(
            data=corr_num,
            annot=True,
            vmin=-1,
            vmax=1,
            fmt=".2f",
            annot_kws={"size": my_font_size}
        )
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ Not enough numeric attributes for a correlation matrix.")

    # ALL VARIABLES HEATMAP
    corr_matrix = df_S7[numeric_att + category_att].corr()
    corr_order = corr_matrix.mean().sort_values(ascending=False).index
    corr_matrix = corr_matrix.loc[corr_order, corr_order]

    fig = plt.figure(figsize=(2 * figWidth_unit, 2 * figHeight_unit))
    plt.title("CATEGORIC AND NUMERIC ATTRIBUTES", fontsize=font_size_titles + 2, fontweight="bold")
    sns.heatmap(
        data=corr_matrix,
        annot=True,
        vmin=-1,
        vmax=1,
        fmt=".2f",
        annot_kws={"size": my_font_size}
    )
    plt.tight_layout()
    plt.show()

    # PAIRPLOT (sorted by correlation order)
    fig = plt.figure(figsize=(figWidth_unit, figHeight_unit))
    sns.pairplot(data=df_S7[corr_order])
    plt.show()


CONCLUSIONS:
- attributes do not keep a hight correlation between them -> no noise

In [ ]:
print("STEP 8) MISSING VALUES")

# ||||||||||||||||||
# ||||| INPUTS |||||
# ||||||||||||||||||
zero_to_nan = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]                    # List of attributes where zero should be considered missing
filling_threshold = 5.0             # [%] If missing perc > filling_threshold → fill values, otherwise drop rows
grouping_max_unique = 6             # Max number of unique values for a categorical attribute to be usable as keys for grouped median
make_missing_values_plots = True    # Make plots?

# Copy of previous DataFrame
df_S8 = df_S4.copy()

# Replace zeros by NaN for selected columns
for col in zero_to_nan:
    if col in df_S8.columns:
        df_S8[col] = df_S8[col].replace(0, np.nan)
        print(f"- ⚠️ Values equal to 0 in '{col}' have been replaced by NaN")

# TARGET VARIABLE
missing_y = df_S8[y_var].isnull().sum()

if missing_y > 0:
    print(f"- ⚠️ Target variable '{y_var}' contains {missing_y} missing values → rows will be dropped.")
    df_S8 = df_S8.dropna(subset=[y_var])
else:
    print(f"- ✅ Target variable '{y_var}' has no missing values.")

# Identify categorical variables usable as grouping keys for numeric imputation
group_vars = []

# Normal categorical attributes
for col in category_att:
    if df_S8[col].nunique() <= grouping_max_unique:
        group_vars.append(col)

# Add target as grouping variable if it is CATEGORICAL and has few unique values
if y_var_type == "CATEGORIC":
    if df_S8[y_var].nunique() <= grouping_max_unique:
        group_vars.append(y_var)
        print(f"- ℹ️ Target variable '{y_var}' added to grouping keys for numeric imputation")

# Calculate missing percentages per column
missing_pct = (df_S8.isnull().sum() / len(df_S8)) * 100
missing_pct = missing_pct[missing_pct > 0].sort_values(ascending=False)

if len(missing_pct) == 0:
    print("- ✅ DataFrame has no missing values at all (excluding target variable already handled)")
else:
    # Process each column with missing values
    for col in missing_pct.index:
        pct = missing_pct[col]
        print(f"- ⚠️ Column: {col} → {pct:.2f}% missing")

        # CASE 1: NUMERIC ATTRIBUTE
        if col in numeric_att:

            # CASE 1A: grouped median
            if pct > filling_threshold and len(group_vars) > 0:
                print(f" - ⚠️ FILLED missing numeric values in {col} using grouped median by {group_vars}...\n")

                medians = df_S8.groupby(group_vars)[col].median().reset_index()
                medians = medians.rename(columns={col: f"median_{col}"})

                df_S8 = pd.merge(df_S8, medians, on=group_vars, how="left")
                df_S8[col] = df_S8[col].fillna(df_S8[f"median_{col}"])
                df_S8 = df_S8.drop(columns=[f"median_{col}"])

            # CASE 1B: global median
            elif pct > filling_threshold and len(group_vars) == 0:
                print(f" - ⚠️ FILLED missing numeric values in {col} using global median (no grouping columns)...\n")
                df_S8[col] = df_S8[col].fillna(df_S8[col].median())

            # CASE 1C: drop rows
            elif pct <= filling_threshold:
                print(f" - ⚠️ DROPPED rows with missing values in {col} ({pct:.2f}% ≤ {filling_threshold}%)...\n")
                df_S8 = df_S8.dropna(subset=[col])

        # CASE 2: CATEGORICAL ATTRIBUTE → mode imputation
        elif col in category_att:

            print(f" - ⚠️ FILLED missing categorical values in {col} using mode (most frequent value)...\n")
            mode_value = df_S8[col].mode().iloc[0]
            df_S8[col] = df_S8[col].fillna(mode_value)

        # CASE 3: unsupported
        else:
            print(f" - ℹ️ Column {col} has unsupported type for imputation — dropping rows.\n")
            df_S8 = df_S8.dropna(subset=[col])

# Print results
print("- ✅ Missing values have been handled successfully!")
print(f" - ℹ️ Previous df's rows: {len(df_S4)}")
print(f" - ℹ️ Current df's rows: {len(df_S8)}")
print(f" - ℹ️ Current DataFrame shape: {df_S8.shape}")
print(f" - ℹ️ Remaining missing values per column:\n{df_S8.isnull().sum()}")

if make_missing_values_plots:
    # BEFORE vs AFTER missing values handling
    print("\n📊 VISUAL CHECK - BEFORE vs AFTER missing values handling")

    df_S8_before = df_S4.copy()   # Before missing-value handling
    df_S8_after = df_S8.copy()    # After missing-value handling

    if not numeric_att:
        print("   This type of plot is non applicable for this case, because there are not NUMERIC variables in the DataFrame")

    else:
        var_to_plot = numeric_att.copy()
        if y_var_type == "NUMERIC" and y_var not in var_to_plot:
            var_to_plot.insert(0, y_var)

        num_rows = len(var_to_plot)

        fig, axes = plt.subplots(
            nrows = num_rows * 2,
            ncols = 2,
            figsize = (figWidth_unit * 2, figHeight_unit * num_rows),
            gridspec_kw={'height_ratios': [4, 0.5] * num_rows})

        for i, colname in enumerate(var_to_plot):

            # Row indices for histogram and boxplot of this variable
            hist_row  = i * 2
            box_row   = i * 2 + 1

            # Common bins (syncronize BEFORE and AFTER)
            xmin = min(df_S8_before[colname].min(), df_S8_after[colname].min())
            xmax = max(df_S8_before[colname].max(), df_S8_after[colname].max())
            common_bins = np.linspace(xmin, xmax, num_bins + 1)

            # ================
            # BEFORE PLOTS
            # ================
            before_hist_ax = axes[hist_row, 0]
            before_box_ax  = axes[box_row, 0]

            sns.histplot(
                ax = before_hist_ax,
                data = df_S8_before,
                x = colname,
                bins = num_bins,
                color = "gray",
                alpha = 0.35)
            before_hist_ax.set_title(colname + " - BEFORE")
            before_hist_ax.set_xlabel("")

            sns.boxplot(
                ax = before_box_ax,
                data = df_S8_before,
                x = colname,
                color = "lightgray")
            before_box_ax.set_xlabel("")

            # Store BEFORE axis limits
            xlim_hist_before = before_hist_ax.get_xlim()
            ylim_hist_before = before_hist_ax.get_ylim()
            xlim_box_before  = before_box_ax.get_xlim()

            # ================
            # AFTER PLOTS
            # ================
            after_hist_ax = axes[hist_row, 1]
            after_box_ax  = axes[box_row, 1]

            sns.histplot(
                ax = after_hist_ax,
                data = df_S8_after,
                x = colname,
                bins = common_bins)
            after_hist_ax.set_title(colname + " - AFTER")
            after_hist_ax.set_xlabel("")

            sns.boxplot(
                ax = after_box_ax,
                data = df_S8_after,
                x = colname)
            after_box_ax.set_xlabel("")

            # Syncronize axes limits
            after_hist_ax.set_xlim(xlim_hist_before)
            after_hist_ax.set_ylim(ylim_hist_before)
            after_box_ax.set_xlim(xlim_box_before)

        plt.tight_layout()
        plt.show()

In [ ]:
# -------------------------------
# STEP 9) CLEAN OUTLIERS
# -------------------------------
print("STEP 9) CLEAN OUTLIERS")

# ||||||||||||||||||
# ||||| INPUTS |||||
# ||||||||||||||||||
low_outliers_threshold = 1.0        # [%] Max percentage of lower outliers allowed to remove
up_outliers_threshold = 1.0         # [% ]Max percentage of upper outliers allowed to remove
removal_type = "EXTREME OUTLIERS"   # Removal logic type (NORMAL or EXTREME outliers)
make_outliers_plots = True          # Make plots?

# Copy  of previous DataFrame
df_S9=df_S8.copy()

# Print info
display(df_S9.describe())

# Outliers detection is applied over numeric attributes
cols = []
for col in numeric_att:
    if col in df_S9.columns.to_list():
        cols.append(col)

# Initialize containers
lower_limits = []
upper_limits = []
n_outliers_lower = []
n_outliers_upper = []
pct_outliers_lower = []
pct_outliers_upper = []
extreme_lower_limits = []
extreme_upper_limits = []
n_extreme_outliers_lower = []
n_extreme_outliers_upper = []
pct_extreme_outliers_lower = []
pct_extreme_outliers_upper = []

len(df_S9.index)

for col in cols:
    Q1 = df_S9[col].quantile(0.25)
    Q3 = df_S9[col].quantile(0.75)
    IQR = Q3 - Q1

    # Calulate limits
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    extreme_lower = Q1 - 3 * IQR
    extreme_upper = Q3 + 3 * IQR

    # Count num of outliers
    n_low = (df_S9[col] < lower).sum()
    n_high = (df_S9[col] > upper).sum()
    n_extreme_low = (df_S9[col] < extreme_lower).sum()
    n_extreme_high = (df_S9[col] > extreme_upper).sum()

    # Percentages of outliers
    pct_low = (n_low / len(df_S9.index)) * 100
    pct_high = (n_high / len(df_S9.index)) * 100
    pct_extreme_low = (n_extreme_low / len(df_S9.index)) * 100
    pct_extreme_high = (n_extreme_high / len(df_S9.index)) * 100

    # Save limits
    lower_limits.append(lower)
    upper_limits.append(upper)
    extreme_lower_limits.append(extreme_lower)
    extreme_upper_limits.append(extreme_upper)

    # Save num of outliers
    n_outliers_lower.append(n_low)
    n_outliers_upper.append(n_high)
    n_extreme_outliers_lower.append(n_extreme_low)
    n_extreme_outliers_upper.append(n_extreme_high)

    # Save percentages of outliers
    pct_outliers_lower.append(pct_low)
    pct_outliers_upper.append(pct_high)
    pct_extreme_outliers_lower.append(pct_extreme_low)
    pct_extreme_outliers_upper.append(pct_extreme_high)


# Build DataFrame with all results
df_limits = pd.DataFrame(
    [
        lower_limits,
        upper_limits,
        n_outliers_lower,
        n_outliers_upper,
        pct_outliers_lower,
        pct_outliers_upper
    ],
    columns=cols,
    index=[
        "lower_limit",
        "upper_limit",
        "n_outliers_lower",
        "n_outliers_upper",
        "pct_outliers_lower",
        "pct_outliers_upper"
    ]
)
df_extreme_limits = pd.DataFrame(
    [
        extreme_lower_limits,
        extreme_upper_limits,
        n_extreme_outliers_lower,
        n_extreme_outliers_upper,
        pct_extreme_outliers_lower,
        pct_extreme_outliers_upper
    ],
    columns=cols,
    index=[
        "extreme_lower_limit",
        "extreme_upper_limit",
        "n_extreme_outliers_lower",
        "n_extreme_outliers_upper",
        "pct_extreme_outliers_lower",
        "pct_extreme_outliers_upper"
    ]
)
# Display results
display(df_limits)
display(df_extreme_limits)

# Outliers detection is applied over numeric attributes
cols = []
for col in numeric_att:
    if col in df_S9.columns.to_list():
        cols.append(col)

if removal_type == "NORMAL OUTLIERS":
    # Loop through each column and apply filtering rules
    for col in cols:
        low_limit = df_limits.loc["lower_limit", col]
        high_limit = df_limits.loc["upper_limit", col]
        pct_low = df_limits.loc["pct_outliers_lower", col]
        pct_high = df_limits.loc["pct_outliers_upper", col]
        
        # Remove low outliers if below threshold
        print(f"\n- ATTRIBUTE {col}:")
        if pct_low == 0:
            print(f" - ℹ️ None lower outliers detected")
        elif pct_low <= low_outliers_threshold:
            df_S9 = df_S9[df_S9[col] >= low_limit]
            print(f" - ⚠️ REMOVED lower outliers ({pct_low:.2f}% <= {low_outliers_threshold}%)")
        else:
            print(f" - ✅ KEPT lower outliers ({pct_low:.2f}% > {low_outliers_threshold}%)")
        
        # Remove high outliers if below threshold
        if pct_high == 0:
            print(f" - ℹ️ None upper outliers detected")
        elif pct_high <= up_outliers_threshold:
            df_S9 = df_S9[df_S9[col] <= high_limit]
            print(f" - ⚠️ REMOVED upper outliers ({pct_high:.2f}% <= {up_outliers_threshold}%)")
        else:
            print(f" - ✅ KEPT upper outliers ({pct_high:.2f}% > {up_outliers_threshold}%)")

elif removal_type == "EXTREME OUTLIERS":
    # Loop through each column and apply filtering rules
    for col in cols:
        low_limit = df_extreme_limits.loc["extreme_lower_limit", col]
        high_limit = df_extreme_limits.loc["extreme_upper_limit", col]
        pct_low = df_extreme_limits.loc["pct_extreme_outliers_lower", col]
        pct_high = df_extreme_limits.loc["pct_extreme_outliers_upper", col]
        
        # Remove low outliers if below threshold
        print(f"\n- ATTRIBUTE {col}:")
        if pct_low == 0:
            print(f" - ℹ️ None extreme lower outliers detected")
        elif pct_low <= low_outliers_threshold:
            df_S9 = df_S9[df_S9[col] >= low_limit]
            print(f" - ⚠️ REMOVED extreme lower outliers ({pct_low:.2f}% <= {low_outliers_threshold}%)")
        else:
            print(f" - ✅ KEPT extreme lower outliers ({pct_low:.2f}% > {low_outliers_threshold}%)")
        
        # Remove high outliers if below threshold
        if pct_high == 0:
            print(f" - ℹ️ None extreme upper outliers detected")
        elif pct_high <= up_outliers_threshold:
            df_S9 = df_S9[df_S9[col] <= high_limit]
            print(f" - ⚠️ REMOVED extreme upper outliers ({pct_high:.2f}% <= {up_outliers_threshold}%)")
        else:
            print(f" - ✅ KEPT extreme upper outliers ({pct_high:.2f}% > {up_outliers_threshold}%)")

# Print results
print("- ✅ Outliers have been handled successfully!")
print(f" - ℹ️ Previous df's rows: {len(df_S8)}")
print(f" - ℹ️ Current df's rows: {len(df_S9)}")
print(f" - ℹ️ Current DataFrame shape: {df_S9.shape}")
display(df_S9.describe())

if make_outliers_plots:
    # BEFORE vs AFTER Outliers handling
    print("\n📊 VISUAL CHECK - BEFORE vs AFTER outliers handling")

    df_S9_before = df_S8.copy()   # Before missing-value handling
    df_S9_after = df_S9.copy()    # After missing-value handling

    if not numeric_att:
        print("   This type of plot is non applicable for this case, because there are not NUMERIC variables in the DataFrame")

    else:
        var_to_plot = numeric_att
        num_rows = len(var_to_plot)

        fig, axes = plt.subplots(
            nrows = num_rows * 2,
            ncols = 2,
            figsize = (figWidth_unit * 2, figHeight_unit * num_rows),
            gridspec_kw={'height_ratios': [4, 0.5] * num_rows})

        for i, colname in enumerate(var_to_plot):

            # Row indices for histogram and boxplot of this variable
            hist_row  = i * 2
            box_row   = i * 2 + 1

            # Set common bins (syncronize BEFORE and AFTER)
            xmin = min(df_S9_before[colname].min(), df_S9_after[colname].min())
            xmax = max(df_S9_before[colname].max(), df_S9_after[colname].max())
            common_bins = np.linspace(xmin, xmax, num_bins + 1)

            # Set colored area limits
            normal_low = df_limits.loc["lower_limit", colname]
            normal_up  = df_limits.loc["upper_limit", colname]
            extreme_low = df_extreme_limits.loc["extreme_lower_limit", colname]
            extreme_up  = df_extreme_limits.loc["extreme_upper_limit", colname]

            # ================
            # BEFORE PLOTS
            # ================
            before_hist_ax = axes[hist_row, 0]
            before_box_ax  = axes[box_row, 0]

            sns.histplot(
                ax = before_hist_ax,
                data = df_S9_before,
                x = colname,
                bins = num_bins,
                color = "gray",
                alpha = 0.35)
            before_hist_ax.set_title(colname + " - BEFORE")
            before_hist_ax.set_xlabel("")

            sns.boxplot(
                ax = before_box_ax,
                data = df_S9_before,
                x = colname,
                color = "lightgray")
            before_box_ax.set_xlabel("")

            # Outlier count
            pct_low_normal  = df_limits.loc["pct_outliers_lower", colname]
            pct_high_normal = df_limits.loc["pct_outliers_upper", colname]
            pct_low_extreme  = df_extreme_limits.loc["pct_extreme_outliers_lower", colname]
            pct_high_extreme = df_extreme_limits.loc["pct_extreme_outliers_upper", colname]

            # NORMAL Outliers
            if pct_low_normal > 0:
                before_hist_ax.axvspan(normal_low, extreme_low, color="orange", alpha=0.22)
                before_box_ax.axvspan(normal_low, extreme_low, color="orange", alpha=0.22)
            if pct_high_normal > 0:
                before_hist_ax.axvspan(extreme_up, normal_up, color="orange", alpha=0.22)
                before_box_ax.axvspan(extreme_up, normal_up, color="orange", alpha=0.22)

            # EXTREME Outliers
            if pct_low_extreme > 0:
                before_hist_ax.axvspan(xmin, extreme_low, color="red", alpha=0.22)
                before_box_ax.axvspan(xmin, extreme_low, color="red", alpha=0.22)
            if pct_high_extreme > 0:
                before_hist_ax.axvspan(extreme_up, xmax, color="red", alpha=0.22)
                before_box_ax.axvspan(extreme_up, xmax, color="red", alpha=0.22)

            # Store BEFORE limits
            xlim_hist_before = before_hist_ax.get_xlim()
            ylim_hist_before = before_hist_ax.get_ylim()
            xlim_box_before  = before_box_ax.get_xlim()

            # ================
            # AFTER PLOTS
            # ================
            after_hist_ax = axes[hist_row, 1]
            after_box_ax  = axes[box_row, 1]

            sns.histplot(
                ax = after_hist_ax,
                data = df_S9_after,
                x = colname,
                bins = common_bins)
            after_hist_ax.set_title(colname + " - AFTER")
            after_hist_ax.set_xlabel("")

            sns.boxplot(
                ax = after_box_ax,
                data = df_S9_after,
                x = colname)
            after_box_ax.set_xlabel("")

            # Check if outliers are still present in AFTER
            normal_low_present  = (df_S9_after[colname] < normal_low).any()
            normal_up_present   = (df_S9_after[colname] > normal_up).any()
            extreme_low_present = (df_S9_after[colname] < extreme_low).any()
            extreme_up_present  = (df_S9_after[colname] > extreme_up).any()

            # NORMAL Outliers
            if normal_low_present:
                after_hist_ax.axvspan(normal_low, extreme_low, color="orange", alpha=0.22)
                after_box_ax.axvspan(normal_low, extreme_low, color="orange", alpha=0.22)
            if normal_up_present:
                after_hist_ax.axvspan(extreme_up, normal_up, color="orange", alpha=0.22)
                after_box_ax.axvspan(extreme_up, normal_up, color="orange", alpha=0.22)

            # EXTREME Outliers
            if extreme_low_present:
                after_hist_ax.axvspan(xmin, extreme_low, color="red", alpha=0.22)
                after_box_ax.axvspan(xmin, extreme_low, color="red", alpha=0.22)
            if extreme_up_present:
                after_hist_ax.axvspan(extreme_up, xmax, color="red", alpha=0.22)
                after_box_ax.axvspan(extreme_up, xmax, color="red", alpha=0.22)

            # Legends
            before_hist_ax.legend(
                handles=[
                    plt.Rectangle((0,0),1,1, facecolor='orange', alpha=0.45, label="NORMAL OUTLIERS"),
                    plt.Rectangle((0,0),1,1, facecolor='red', alpha=0.45, label="EXTREME OUTLIERS")],
                loc="upper right")
            after_hist_ax.legend(
                handles=[
                    plt.Rectangle((0,0),1,1, facecolor='orange', alpha=0.45, label="NORMAL OUTLIERS"),
                    plt.Rectangle((0,0),1,1, facecolor='red', alpha=0.45, label="EXTREME OUTLIERS")],
                loc="upper right")

            # Syncronize axes limits
            after_hist_ax.set_xlim(xlim_hist_before)
            after_hist_ax.set_ylim(ylim_hist_before)
            after_box_ax.set_xlim(xlim_box_before)

        plt.tight_layout()
        plt.show()

CONCLUSIONS:
- I have stablished a "cut" based on EXTREME outliers
- The "cut" would be for a maximum of 1 %, higher that than that, EXTREME outliers will be kept

In [ ]:
# -------------------------------
# STEP 10) REMOVE NOISY ATTRIBUTES
# -------------------------------
print("STEP 10) REMOVE NOISY ATTRIBUTES")

# ||||||||||||||||||
# ||||| INPUTS |||||
# ||||||||||||||||||
corr_threshold = 0.9 # Correlation level considered as "too high"

# Copy of previous DataFrame
df_S10 = df_S9.copy()

#  NUMERIC ATTRIBUTES (Pearson correlation)
corr_matrix = df_S10[numeric_att].corr().abs()
to_drop = set()

for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if corr_matrix.iloc[i, j] > corr_threshold:
            col_i = corr_matrix.columns[i]
            col_j = corr_matrix.columns[j]
            if col_i not in to_drop:
                to_drop.add(col_i)

if to_drop:
    df_S10 = df_S10.drop(columns=list(to_drop), axis=1)
    print(f"- ⚠️ High NUMERIC attributes correlation detected (Pearson Corr. > {corr_threshold}). Dropped:")
    for col in to_drop:
        print(f"   • {col}")
else:
    print(f"- ✅ No NUMERIC attributes exceeded {corr_threshold} Pearson Correlation")

#  CATEGORICAL ATTRIBUTES (Cramér's V)
def cramers_v(x, y): 
    # Step 1: confusion matrix
    confusion_matrix = pd.crosstab(x, y)
    # Step 2: chi-square statistic
    chi2 = chi2_contingency(confusion_matrix)[0]
    # Step 3: phi-squared
    total_samples = confusion_matrix.sum().sum()
    phi2 = chi2 / total_samples
    # Shape of confusion matrix
    r, k = confusion_matrix.shape
    num_rows = confusion_matrix.shape[0]
    num_cols = confusion_matrix.shape[1]
    # Step 4: bias correction (recommended formula)
    correction = ((num_cols - 1) * (num_rows - 1)) / (total_samples - 1)
    phi2_corrected = max(0, phi2 - correction)
    # Corrected dimensions
    rows_corrected = num_rows - ((num_rows - 1) ** 2) / (total_samples - 1)
    cols_corrected = num_cols - ((num_cols - 1) ** 2) / (total_samples - 1)
    # Step 5: compute Cramér's V
    denominator = min(rows_corrected - 1, cols_corrected - 1)
    if denominator <= 0:
        return 0  # avoid division by zero for degenerate tables
    cramers_v_value = np.sqrt(phi2_corrected / denominator)
    return cramers_v_value

to_drop_cat = set()

if len(category_att) > 1:
    for i in range(len(category_att)):
        for j in range(i):
            v = cramers_v(df_S10[category_att[i]], df_S10[category_att[j]])
            if v > corr_threshold:
                col_i = category_att[i]
                col_j = category_att[j]
                if col_i not in to_drop_cat:
                    to_drop_cat.add(col_i)

if to_drop_cat:
    df_S10 = df_S10.drop(columns=list(to_drop_cat), axis=1)
    print(f"- ⚠️ High CATEGORICAL attributes association detected (Cramer’s V > {corr_threshold}). Dropped:")
    for col in to_drop_cat:
        print(f"   • {col}")
else:
    print(f"- ✅ No CATEGORICAL attributes exceeded {corr_threshold} Cramer’s V")

# Update numeric_att
updated_numeric = []
for col in numeric_att:
    if col in df_S10.columns:
        updated_numeric.append(col)
numeric_att = updated_numeric

# Update category_att
updated_category = []
for col in category_att:
    if col in df_S10.columns:
        updated_category.append(col)
category_att = updated_category

# Update binary_att
updated_binary = []
for col in binary_att:
    if col in df_S10.columns:
        updated_binary.append(col)
binary_att = updated_binary

# Update multiclass_att
updated_multiclass = []
for col in multiclass_att:
    if col in df_S10.columns:
        updated_multiclass.append(col)
multiclass_att = updated_multiclass

#  Print results
print(f"- ℹ️ Previous df's columns: {len(df_S9.columns)}")
print(f"- ℹ️ Cleaned df's  columns: {len(df_S10.columns)}")
print(f"- ℹ️ Final DataFrame shape: {df_S10.shape}")
display(df_S10.head())

In [ ]:
# -------------------------------
# STEP 11) SPLIT
# -------------------------------
print("STEP 11) SPLIT")

# ||||||||||||||||||
# ||||| INPUTS |||||
# ||||||||||||||||||
my_test_size = 0.2
random_seed = 42

# Copy  of previous DataFrame
df_SPLIT = df_S10.copy()

# Separate attributes from target variable
X = df_SPLIT.drop(labels = y_var, axis = 1)
y = df_SPLIT[y_var]

# Make split between Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = my_test_size, random_state = random_seed)

print("- ℹ️ Shape of DataFrames after SPLIT:")
print(" - X_train:",X_train.shape)
print(" - X_test:",X_test.shape)
print(" - y_train:",y_train.shape)
print(" - y_test:",y_test.shape)

print("\n- ℹ️ Content of DataFrames after SPLIT:")
print(" - X_train:\n",X_train.head(5))
print(" - X_test:\n",X_test.head(5))
print(" - y_train:\n",y_train.head(5))
print(" - y_test:\n",y_test.head(5))

print("\n- ℹ️ Info of DataFrames after SPLIT:")
print(" - X_train:",X_train.info())
print(" - X_test:",X_test.info())
print(" - y_train:",y_train.info())
print(" - y_test:",y_test.info())

In [ ]:
# -------------------------------
# STEP 12) SCALLING
# -------------------------------
print("STEP 12) SCALLING")

# ||||||||||||||||||
# ||||| INPUTS |||||
# ||||||||||||||||||
make_scalling = False
scaler_dic={# Instance scaler for each numeric attribute - change manually StandardScaler() or MinMaxScaler()
}
for col in X_train.columns:
    if col in numeric_att:
        scaler_dic[col] = StandardScaler()

# Copy of previous DataFrame
X_train_es = X_train.copy()
X_test_es = X_test.copy()

if not numeric_att:
    X_train_es = X_train_es[numeric_att]
    X_test_es = X_test_es[numeric_att]
    print("  ⚠️ SCALLING is non applicable for this case, because there are not NUMERIC attributes in the DataFrame")
elif not make_scalling:
    X_train_es = X_train_es[numeric_att]
    X_test_es = X_test_es[numeric_att]
    print("  ⚠️ SCALLING is is not carried out, set make_scalling = True")
    display(X_train_es.head())
else:

    # Fit scalers ONLY on train data
    for col in numeric_att:
        scaler_dic[col].fit(X_train_es[[col]])
    print("- ✅ All Scalers have been trained successfully")

    # Apply scalers and create scaled columns
    scaled_cols = []

    for col in numeric_att:
        # Detect scaler type
        scaler_name = scaler_dic[col].__class__.__name__
        # Set suffix
        if scaler_name == "StandardScaler":
            suffix = "_SS"
        elif scaler_name == "MinMaxScaler":
            suffix = "_MM"
        else:
            suffix = "_Scaled"

        # Transform
        X_train_es[col + suffix] = scaler_dic[col].transform(X_train_es[[col]])
        X_test_es[col + suffix]  = scaler_dic[col].transform(X_test_es[[col]])

        scaled_cols.append(col + suffix)
        print(f"- ✅ Train/Test scaled for: {col} using {scaler_name} → new column: {col + suffix}")

    # Keep only scaled columns
    X_train_es = X_train_es[scaled_cols]
    X_test_es  = X_test_es[scaled_cols]

    print("- ✅ Final scaled datasets created successfully!")
    display(X_train_es.head())

CONCLUSION:
- I do not make scalling beause Random Forest is an algorithm that does not need scalled data

In [ ]:
# -------------------------------
# STEP 13) ENCODING
# -------------------------------
print("STEP 13) ENCODING")

# ||||||||||||||||||
# ||||| INPUTS |||||
# ||||||||||||||||||
make_encoding = True

# Copy of previous DataFrame
X_train_en = X_train.copy()
X_test_en = X_test.copy()

if not category_att:
    X_train_en = X_train_en[category_att]
    X_test_en = X_test_en[category_att]
    print("  ⚠️ ENCODING is non applicable for this case, because there are not CATEGORIC attributes in the DataFrame")
elif not make_encoding:
    X_train_en = X_train_en[category_att]
    X_test_en = X_test_en[category_att]
    print("  ⚠️ ENCODING is is not carried out, set make_encoding = True")
    display(X_train_en.head())
else:
    # List of columns
    columns = X_train_en.columns.tolist()

    # Create encoder instance for each categorical attribute
    encoder_dic = {}
    for col in category_att:
        if col in binary_att:
            encoder_dic[col] = LabelEncoder()
            print(f"- ✅ Encoder instanced successfully for {col}: LabelEncoder()")

        elif col in multiclass_att:
            encoder_dic[col] = OneHotEncoder(sparse_output=False)
            print(f"- ✅ Encoder instanced successfully for {col}: OneHotEncoder()")

    # Train encoders with TRAIN data only
    for col in category_att:
        encoder = encoder_dic[col]
        if isinstance(encoder, LabelEncoder):
            encoder.fit(X_train_en[col])        # LabelEncoder needs 1D
            print(f"- ✅ Encoder trained successfully with {col} from Train: LabelEncoder()")

        elif isinstance(encoder, OneHotEncoder):
            encoder.fit(X_train_en[[col]])      # OHE needs 2D
            print(f"- ✅ Encoder trained successfully with {col} from Train: OneHotEncoder()")

    # Apply encoders to TRAIN + TEST
    for col in category_att:
        encoder = encoder_dic[col]
        if isinstance(encoder, LabelEncoder):
            X_train_en[col + "_LE"] = encoder.transform(X_train_en[col])
            X_test_en[col + "_LE"] = encoder.transform(X_test_en[col])
            print(f"- ✅ Train/Test encoded for: {col} using LabelEncoder()")

        elif isinstance(encoder, OneHotEncoder):

            # Transform train and test
            train_encoded = encoder.transform(X_train_en[[col]])
            test_encoded = encoder.transform(X_test_en[[col]])

            # New names
            ohe_colnames = encoder.get_feature_names_out([col])
            ohe_colnames = [name + "_OHE" for name in ohe_colnames]

            # Convert to DataFrames
            train_ohe_df = pd.DataFrame(train_encoded, index=X_train_en.index, columns=ohe_colnames)
            test_ohe_df = pd.DataFrame(test_encoded, index=X_test_en.index, columns=ohe_colnames)

            # Concatenate new cols
            X_train_en = pd.concat([X_train_en, train_ohe_df], axis=1)
            X_test_en = pd.concat([X_test_en, test_ohe_df], axis=1)

            print(f"- ✅ Train/Test encoded for: {col} using OneHotEncoder()")

    # Keep only encoded columns
    encoded_cols = []
    for col in category_att:
        encoder = encoder_dic[col]
        if isinstance(encoder, LabelEncoder):
            encoded_cols.append(col + "_LE")
        elif isinstance(encoder, OneHotEncoder):
            ohe_colnames = encoder.get_feature_names_out([col])
            for name in ohe_colnames:
                encoded_cols.append(name + "_OHE")

    X_train_en = X_train_en[encoded_cols]
    X_test_en = X_test_en[encoded_cols]

    print("- ✅ Final encoded datasets created successfully")
    display(X_train_en.head())

In [ ]:
# -------------------------------
# STEP 14) FEATURE SELECTION
# -------------------------------
print("STEP 14) FEATURE SELECTION")

# ||||||||||||||||||
# ||||| INPUTS |||||
# ||||||||||||||||||
feature_keeping_threshold = 100 # [%] Percentaje of features to keep (SelectKBest) 

# Concatenate NUMERIC_var_scaled with CATEGORY_var_encoded
X_train_assembled = pd.concat([X_train_es, X_train_en], axis=1)
X_test_assembled = pd.concat([X_test_es, X_test_en], axis=1)

# Instance selector
num_features_to_keep = round(feature_keeping_threshold/100 * len(X_train_assembled.columns))
selection_model = SelectKBest(score_func = f_classif, k = num_features_to_keep)
print("- ✅ Selector have been instanced successfully to keep " + str(num_features_to_keep) + " features")

# Train selector with ONLY train data (y_train must be included because this is SUPERVISED selector)
selection_model.fit(X_train_assembled, y_train)
print("- ✅ Selector have been trained successfully")

# Drop non-selected features
keeping_mask = selection_model.get_support()
X_train_assembled = pd.DataFrame(selection_model.transform(X_train_assembled), columns = X_train_assembled.columns.values[keeping_mask])
X_test_assembled = pd.DataFrame(selection_model.transform(X_test_assembled), columns = X_test_assembled.columns.values[keeping_mask])

# Print results
print("\n 🧮 X_train_assembled", X_train_assembled.shape)
display(X_train_assembled.head())
print("\n 🧮 X_test_assembled", X_test_assembled.shape)
display(X_test_assembled.head())

In [ ]:
# -------------------------------
# STEP 15) SAVE PROCESSED DATA
# -------------------------------
print("STEP 15) SAVE PROCESSED DATA")

# ||||||||||||||||||
# ||||| INPUTS |||||
# ||||||||||||||||||
output_path = "../data/processed/"

# Set from previous DataFrame
X_train_final = X_train_assembled
X_test_final = X_test_assembled
y_train_final = y_train
y_test_final = y_test

# Get revision number - Returns the next free integer revision based on existing files
import os
def get_revision_number(base_path, base_name):
    rev = 0
    while True:
        full_path = os.path.join(base_path, base_name + "_" + str(rev) + ".csv")
        if not os.path.exists(full_path):
            return rev
        rev += 1

# Build filenames WITH revision number
rev_number = get_revision_number(output_path, "X_train_final")
suffix = "_" + str(rev_number)

output_path_X_train = output_path + "X_train_final" + suffix + ".csv"
output_path_X_test  = output_path + "X_test_final"  + suffix + ".csv"
output_path_y_train = output_path + "y_train_final" + suffix + ".csv"
output_path_y_test  = output_path + "y_test_final"  + suffix + ".csv"

# Save all datasets
X_train_final.to_csv(output_path_X_train, index=False)
X_test_final.to_csv(output_path_X_test, index=False)
y_train_final.to_csv(output_path_y_train, index=False)
y_test_final.to_csv(output_path_y_test, index=False)

print("- ✅ Files saved with revision number:", rev_number)
print("- 💡 Reminder: data/processed folder is ignored in .gitignore")

In [ ]:
# -------------------------------
# STEP 16) PREDICTION MODELS
# -------------------------------
print("STEP 16) PREDICTION MODEL")

# |||||||||||||||||| 
# ||||| INPUTS |||||
# ||||||||||||||||||
rev_to_use = 0                            # Enter the desired revision number
grid_cross_validation = 10                # Parameter cv for GridSearchCV
grid_classification_scoring = "accuracy"  # Choose between: "accuracy", "precision", "recall", "f1"
grid_regression_scoring = "r2"            # Choose between: "neg_root_mean_squared_error", "r2"

classification_models_selection = {
    "LogisticRegression": False,
    "DecisionTreeClassifier": False,
    "RandomForestClassifier": True
}

regression_models_selection = {
    "LinearRegression": False,
    "DecisionTreeRegressor": False,
    "RandomForestRegressor": True,
    "Lasso": False,
    "Ridge": False
}

classification_grids = {
    "LogisticRegression": {
        "penalty": ["l1", "l2", "elasticnet", None],
        "C": [0.001, 0.01, 0.1, 1, 10, 100],
        "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
    },
    "DecisionTreeClassifier": {
        "criterion": ["gini", "entropy", "log_loss"],
        "max_depth": [None, 5, 10, 20, 30],
        "min_samples_split": [2, 5, 10, 20, 30]
    },
    "RandomForestClassifier": {
        "n_estimators": [5, 10, 20, 50, 100, 200],
        "max_depth": [None, 2, 5, 10, 15, 20]
    }
}

regression_grids = {
    "LinearRegression": {},
    "Lasso": {
        "alpha": [0.001, 0.005, 0.01, 0.1, 0.5, 1, 2, 5, 10, 20, 50],
        "max_iter": [1000, 5000, 10000]
    },
    "Ridge": {
        "alpha": [0.001, 0.005, 0.01, 0.1, 0.5, 1, 2, 5, 10, 20, 50],
        "max_iter": [1000, 5000, 10000]
    },
    "DecisionTreeRegressor": {
        "criterion": ["squared_error", "friedman_mse", "absolute_error", "poisson"],
        "max_depth": [None, 5, 10, 20, 30],
        "min_samples_split": [2, 5, 10, 20, 30]
    },
    "RandomForestRegressor": {
        "n_estimators": [5, 10, 20, 50, 100, 200],
        "max_depth": [None, 2, 5, 10, 15, 20]
    }
}

# Load processed data according to selected revision
try:
    X_train_model = pd.read_csv(output_path + "X_train_final_" + str(rev_to_use) + ".csv")
    X_test_model  = pd.read_csv(output_path + "X_test_final_" + str(rev_to_use) + ".csv")
    y_train_model = pd.read_csv(output_path + "y_train_final_" + str(rev_to_use) + ".csv").squeeze()
    y_test_model  = pd.read_csv(output_path + "y_test_final_" + str(rev_to_use) + ".csv").squeeze()

except FileNotFoundError as e:
    raise FileNotFoundError(
        f"❌ The selected revision rev={rev_to_use} does NOT exist.\n"
        f"Missing file: {e.filename}\n"
        f"Please check available revision numbers in '../data/processed/'."
    )
print(f"➡️ Loaded revision {rev_to_use}")

# Available models
classification_available = {
    "LogisticRegression": LogisticRegression(random_state=random_seed),
    "RandomForestClassifier": RandomForestClassifier(random_state=random_seed),
    "DecisionTreeClassifier": DecisionTreeClassifier(random_state=random_seed)
}

regression_available = {
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(random_state=random_seed),
    "Ridge": Ridge(random_state=random_seed),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=random_seed),
    "RandomForestRegressor": RandomForestRegressor(random_state=random_seed)
}

#  Auxiliary functions
def compute_classification_metrics(y_true, y_pred, avg, pos_label):
    metrics = {}
    metrics["Accuracy"]  = accuracy_score(y_true, y_pred)
    metrics["Precision"] = precision_score(y_true, y_pred, average=avg, pos_label=pos_label)
    metrics["Recall"]    = recall_score(y_true, y_pred, average=avg, pos_label=pos_label)
    metrics["F1_score"]  = f1_score(y_true, y_pred, average=avg, pos_label=pos_label)
    return metrics

def compute_regression_metrics(y_true, y_pred):
    metrics = {}
    metrics["MAE"]  = mean_absolute_error(y_true, y_pred)
    metrics["MSE"]  = mean_squared_error(y_true, y_pred)
    metrics["RMSE"] = np.sqrt(metrics["MSE"])
    metrics["R2"]   = r2_score(y_true, y_pred)
    return metrics

def set_average_proposal(y):
    unique_count = y.nunique()
    if unique_count == 2:
        freq = y.value_counts()
        pos_label = freq.index[-1]
        return "binary", pos_label, grid_classification_scoring

    freq_norm = y.value_counts(normalize=True)
    imbalance_ratio = freq_norm.max() / freq_norm.min()

    if imbalance_ratio <= 1.2:
        return "micro", None, grid_classification_scoring + "_micro" if grid_classification_scoring != "accuracy" else grid_classification_scoring
    if imbalance_ratio <= 1.5:
        return "macro", None, grid_classification_scoring + "_macro" if grid_classification_scoring != "accuracy" else grid_classification_scoring
    return "weighted", None, grid_classification_scoring + "_weighted" if grid_classification_scoring != "accuracy" else grid_classification_scoring

def left_align(df):
    return df.style.set_table_styles(
        [{'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])

# List with selected models
classification_selected_list = []
regression_selected_list = []
for model_name, active in classification_models_selection.items():
    if active:
        classification_selected_list.append(model_name)
for model_name, active in regression_models_selection.items():
    if active:
        regression_selected_list.append(model_name)

# Auto-add Lasso/Ridge if LinearRegression is selected
if "LinearRegression" in regression_selected_list:
    if "Lasso" not in regression_selected_list:
        regression_selected_list.append("Lasso")
    if "Ridge" not in regression_selected_list:
        regression_selected_list.append("Ridge")

print(f"\n - Classification models selected: {classification_selected_list}")
print(f" - Regression models selected:     {regression_selected_list}")

# Parameters needed for classification metrics
if len(classification_selected_list) > 0:
    proposed_avg, proposed_pos, proposed_score = set_average_proposal(y_train_model)

# ======================================================
#  CLASSIFICATION MODELS
# ======================================================
trained_models = {} 
default_results_class = {}
optimized_results_class = {}
for model_name in classification_selected_list:
    # Instance DEFAULT model
    default_model = classification_available[model_name] 
    # Train DEFAULT model
    default_model.fit(X_train_model, y_train_model)
    # Predict with trained DEFAULT model
    y_pred_train = default_model.predict(X_train_model)
    y_pred_test  = default_model.predict(X_test_model)
    # Calculate metricts for DEFAULT model
    metrics_train = compute_classification_metrics(y_train_model, y_pred_train, proposed_avg, proposed_pos)
    metrics_test  = compute_classification_metrics(y_test_model, y_pred_test, proposed_avg, proposed_pos)
    # Build final table with results for DEFAULT model
    default_results_class["DEFAULT " + model_name + " - 🏋️ TRAIN"] = metrics_train
    default_results_class["DEFAULT " + model_name + " - 🧪 TEST"]  = metrics_test
    # Store trained model
    trained_models[model_name] = {"default": default_model, "optimized": None}

    # Set GRID parameters
    grid_params = classification_grids[model_name]
    if len(grid_params) > 0:
        # Instance GRID
        grid = GridSearchCV(
            estimator = classification_available[model_name],
            param_grid = grid_params,
            scoring = proposed_score,
            cv = grid_cross_validation)
        # Train GRID
        grid.fit(X_train_model, y_train_model)
        # Get best estimator configuration (OPTIMIZED model)
        best_model = grid.best_estimator_
        # Predict with trained OPTIMIZED model
        y_train_opt = best_model.predict(X_train_model)
        y_test_opt  = best_model.predict(X_test_model)
        # Calculate metricts for OPTIMIZED model
        metrics_train_opt = compute_classification_metrics(y_train_model, y_train_opt, proposed_avg, proposed_pos)
        metrics_test_opt  = compute_classification_metrics(y_test_model,  y_test_opt,  proposed_avg, proposed_pos)
        # Build final table with results for OPTIMIZED model
        optimized_results_class["OPTIMIZED " + model_name + " - 🏋️ TRAIN"] = {**metrics_train_opt, "Best Parameters": grid.best_params_, "Scoring": grid_classification_scoring}
        optimized_results_class["OPTIMIZED " + model_name + " - 🧪 TEST"]  = {**metrics_test_opt,  "Best Parameters": grid.best_params_, "Scoring": grid_classification_scoring}
        # Store trained model
        trained_models[model_name]["optimized"] = best_model

        # Store hyperparameters for RandomForestClassifier
        if model_name == "RandomForestClassifier":
            # Extract grid search raw values
            est_values  = grid.cv_results_["param_n_estimators"].data.astype(float)
            depth_values = grid.cv_results_["param_max_depth"].data
            scores = grid.cv_results_["mean_test_score"]
            # Unique n_estimators
            est_unique = np.sort(np.unique(est_values))
            # Clean depths (convert to Python None or int)
            cleaned_depths = []
            for d in depth_values:
                if d is None:
                    cleaned_depths.append(None)
                else:
                    cleaned_depths.append(int(d))
            # Unique depths excluding None
            depth_unique = []
            for d in cleaned_depths:
                if d is not None and d not in depth_unique:
                    depth_unique.append(d)
            depth_unique = sorted(depth_unique)
            # Dictionary: key = max_depth → list of mean scores over n_estimators
            depth_score_dict = {}
            for d in depth_unique:
                mean_scores = []
                for e in est_unique:
                    mask = (est_values == e) & (np.array(cleaned_depths, dtype=object) == d)
                    mean_scores.append(scores[mask].mean())
                depth_score_dict[d] = mean_scores
            # Store internally
            trained_models[model_name]["est_list"]   = est_unique
            trained_models[model_name]["depth_list"] = depth_unique
            trained_models[model_name]["depth_score_dict"] = depth_score_dict
    else:
        optimized_results_class["DEFAULT " + model_name + " - 🏋️ TRAIN"] = {**metrics_train, "Best Parameters": "N/A", "Scoring": "N/A"}
        optimized_results_class["DEFAULT " + model_name + " - 🧪 TEST"]  = {**metrics_test,  "Best Parameters": "N/A", "Scoring": "N/A"}

# ======================================================
#  REGRESSION MODELS
# ======================================================
default_results_reg = {}
optimized_results_reg = {}
for model_name in regression_selected_list:
    # Instance DEFAULT model
    default_model = regression_available[model_name]
    # Train DEFAULT model
    default_model.fit(X_train_model, y_train_model)
    # Predict with trained DEFAULT model
    y_pred_train = default_model.predict(X_train_model)
    y_pred_test  = default_model.predict(X_test_model)
    # Calculate metricts for DEFAULT model
    metrics_train = compute_regression_metrics(y_train_model, y_pred_train)
    metrics_test  = compute_regression_metrics(y_test_model, y_pred_test)
    # Build final table with results for DEFAULT model
    default_results_reg["DEFAULT " + model_name + " - 🏋️ TRAIN"] = metrics_train
    default_results_reg["DEFAULT " + model_name + " - 🧪 TEST"]  = metrics_test
    # Store trained model
    trained_models[model_name] = {"default": default_model, "optimized": None}

    # Set GRID parameters
    grid_params = regression_grids[model_name]
    if len(grid_params) > 0:
        # Instance GRID
        grid = GridSearchCV(
            estimator = regression_available[model_name],
            param_grid = grid_params,
            scoring = grid_regression_scoring,
            cv = grid_cross_validation)
        # Train GRID
        grid.fit(X_train_model, y_train_model)
        # Get best estimator configuration (OPTIMIZED model)
        best_model = grid.best_estimator_
        # Predict with trained OPTIMIZED model
        y_train_opt = best_model.predict(X_train_model)
        y_test_opt  = best_model.predict(X_test_model)
        # Calculate metricts for OPTIMIZED model
        metrics_train_opt = compute_regression_metrics(y_train_model, y_train_opt)
        metrics_test_opt  = compute_regression_metrics(y_test_model,  y_test_opt)
        # Build final table with results for OPTIMIZED model
        optimized_results_reg["OPTIMIZED " + model_name + " - 🏋️ TRAIN"] = {**metrics_train_opt, "Best Parameters": grid.best_params_, "Scoring": grid_regression_scoring}
        optimized_results_reg["OPTIMIZED " + model_name + " - 🧪 TEST"]  = {**metrics_test_opt,  "Best Parameters": grid.best_params_, "Scoring": grid_regression_scoring}
        # Store trained model
        trained_models[model_name]["optimized"] = best_model

        # Store hyperparameters for Lasso/Ridge
        if model_name == "Lasso" or model_name == "Ridge":
            # Extract grid search raw values
            alpha_values = grid.cv_results_["param_alpha"].data.astype(float)
            scores = grid.cv_results_["mean_test_score"]
            # Compute the unique alpha values used in the grid (removing repetitions)
            alpha_unique = np.sort(np.unique(alpha_values))
            # Loop through each unique alpha value
            mean_scores = []
            for a in alpha_unique:
                # Create a boolean mask
                mask = (alpha_values == a)
                # Compute the mean score for this alpha
                mean_score = scores[mask].mean()
                # Store the resulting mean score
                mean_scores.append(mean_score)
            # Store internally
            trained_models[model_name]["alpha_list"] = alpha_unique
            trained_models[model_name]["alpha_scores"] = mean_scores

        # Store hyperparameters for RandomForest
        if model_name == "RandomForestRegressor":
            # Extract grid search raw values
            est_values  = grid.cv_results_["param_n_estimators"].data.astype(float)
            depth_values = grid.cv_results_["param_max_depth"].data
            scores = grid.cv_results_["mean_test_score"]
            # Unique values of n_estimators
            est_unique = np.sort(np.unique(est_values))
            # Clean depths (convert to Python None or int)
            cleaned_depths = []
            for d in depth_values:
                if d is None:
                    cleaned_depths.append(None)
                else:
                    cleaned_depths.append(int(d))
            # Unique valid depths (excluding None)
            depth_unique = []
            for d in cleaned_depths:
                if d is not None and d not in depth_unique:
                    depth_unique.append(d)
            depth_unique = sorted(depth_unique)
            # Dictionary: key=max_depth → list of mean scores over n_estimators
            depth_score_dict = {}
            for d in depth_unique:
                mean_scores = []
                for e in est_unique:
                    # Mask selecting rows matching both parameters
                    mask = (est_values == e) & (np.array(cleaned_depths, dtype=object) == d)
                    # Mean score for this (n_estimators, max_depth)
                    mean_scores.append(scores[mask].mean())
                depth_score_dict[d] = mean_scores
            # Store internally
            trained_models[model_name]["est_list"]   = est_unique
            trained_models[model_name]["depth_list"] = depth_unique
            trained_models[model_name]["depth_score_dict"] = depth_score_dict

    else:
        optimized_results_reg["DEFAULT " + model_name + " - 🏋️ TRAIN"] = {**metrics_train, "Best Parameters": "N/A", "Scoring": "N/A"}
        optimized_results_reg["DEFAULT " + model_name + " - 🧪 TEST"]  = {**metrics_test,  "Best Parameters": "N/A", "Scoring": "N/A"}

# ======================================================
#  FINAL TABLES
# ======================================================
if len(classification_selected_list)>0:
    print("\n==================== ⚖️ CLASSIFICATION MODELS COMPARISON ====================")
    display(left_align(pd.DataFrame(default_results_class).T))
    display(left_align(pd.DataFrame(optimized_results_class).T))

if len(regression_selected_list)>0:
    print("\n==================== 📈 REGRESSION MODELS COMPARISON ========================")
    display(left_align(pd.DataFrame(default_results_reg).T))
    display(left_align(pd.DataFrame(optimized_results_reg).T))

# ======================================================
# PLOTTING (DEFAULT vs OPTIMIZED)
# ======================================================

# Subplot template
def subplot_template(nrows, ncols):
    fig, ax = plt.subplots(nrows, ncols, figsize=(figWidth_unit * ncols, figHeight_unit * nrows))
    return fig, ax

# Lasso / Ridge hyperparameter plotter
def plotter_Lasso_Ridge_hparame(model_name, grid_params, grid_scores, ax):
    ax.plot(grid_params, grid_scores, marker="o")
    ax.set_xlabel("Alpha", fontsize=my_font_size)
    ax.set_ylabel(grid_regression_scoring, fontsize=my_font_size)
    ax.set_title(f"{model_name} - Alpha vs Scoring ({grid_regression_scoring})", fontsize=font_size_titles)
    ax.grid(True, linestyle="--", alpha=0.4)

# Decision Tree plotter
def plotter_DecisionTree_model(model, X, ax):
    plot_tree(model, feature_names=X.columns, filled=True, ax=ax)

# Generic RandomForest hyperparameter plotter (Classifier + Regressor)
def plotter_RandomForest_hparam(model_name, est_list, depth_list, depth_score_dict, ax):
    # Choose scoring label depending on model type
    if model_name == "RandomForestClassifier":
        scoring_label = grid_classification_scoring
    else:
        scoring_label = grid_regression_scoring
    # Create a color palette (one color per n_estimators)
    palette_colors = sns.color_palette("bright", len(est_list))
    # One curve per n_estimators
    for i, est in enumerate(est_list):
        # Retrieve scores for this n_estimators across all depths
        est_scores = []
        for d in depth_list:
            est_scores.append(depth_score_dict[d][i])
        # Plot curve: X = depth_list, Y = scores
        ax.plot(depth_list, est_scores, marker="o", color=palette_colors[i], label=f"No. trees = {int(est)}")
    # Labels and formatting
    ax.set_xlabel("Max depth", fontsize=my_font_size)
    ax.set_ylabel(scoring_label, fontsize=my_font_size)
    ax.set_title(f"{model_name} - Hyperparameter Analysis", fontsize=font_size_titles)
    ax.grid(True, linestyle="--", alpha=0.4)
    # Add legend
    ax.legend()

# Registry of plotters
model_plotters = {}
model_plotters["Lasso"] = plotter_Lasso_Ridge_hparame
model_plotters["Ridge"] = plotter_Lasso_Ridge_hparame
model_plotters["DecisionTreeClassifier"] = plotter_DecisionTree_model
model_plotters["DecisionTreeRegressor"]  = plotter_DecisionTree_model
model_plotters["RandomForestClassifier"] = plotter_RandomForest_hparam
model_plotters["RandomForestRegressor"] = plotter_RandomForest_hparam

# ======================================================
# EXECUTE PLOTS FOR MODELS THAT SUPPORT IT
# ======================================================

for model_name, model_dict in trained_models.items():
    # Only plot models that have a registered plotter
    if model_name not in model_plotters:
        continue

    print(f"\n==================== 📊 MODEL PLOT: {model_name} ====================")

    # If Lasso or Ridge → use Lasso / Ridge hyperparameter plotter
    if model_name in ["Lasso", "Ridge"]:
        # Set values 
        alphas  = model_dict.get("alpha_list", None)
        scores  = model_dict.get("alpha_scores", None)
        if alphas is not None and scores is not None:
            # Set subplot
            fig, ax = subplot_template(nrows=1, ncols=1)
            # Plot
            plotter_Lasso_Ridge_hparame(model_name=model_name, grid_params=alphas, grid_scores=scores, ax=ax)
            plt.tight_layout()
            plt.show()
        continue

    # If DecisionTree → use Decision Tree plotter
    if model_name in ["DecisionTreeClassifier", "DecisionTreeRegressor"]:
        # Set subplot
        fig, axes = subplot_template(nrows=1, ncols=2)
        # Plot DEFAULT
        plotter_DecisionTree_model(model=model_dict["default"], X=X_train_model, ax=axes[0])
        axes[0].set_title(f"{model_name} - DEFAULT", fontsize=font_size_titles)
        # Plot OPTIMIZED
        plotter_DecisionTree_model(model=model_dict["optimized"], X=X_train_model, ax=axes[1])
        axes[1].set_title(f"{model_name} - OPTIMIZED", fontsize=font_size_titles)
        plt.tight_layout()
        plt.show()
        continue

    # If RandomForest → use unified RandomForest hyperparameter plotter
    if model_name in ["RandomForestClassifier", "RandomForestRegressor"]:
        # Set values 
        est_list   = model_dict.get("est_list", None)
        depth_list = model_dict.get("depth_list", None)
        depth_score_dict = model_dict.get("depth_score_dict", None)
        if est_list is not None and depth_list is not None and depth_score_dict is not None:
            # Set subplot
            fig, ax = subplot_template(nrows=1, ncols=1)
            # Plot
            plotter_RandomForest_hparam(model_name=model_name, est_list=est_list, depth_list=depth_list, depth_score_dict=depth_score_dict, ax=ax)
            plt.tight_layout()
            plt.show()
        continue

In [ ]:
# -------------------------------
# STEP 17) SAVE MODELS
# -------------------------------
print("STEP 17) SAVE MODELS")

# |||||||||||||||||| 
# ||||| INPUTS |||||
# ||||||||||||||||||
models_output_path = "../models/"   # Folder where models will be saved

# Iterate over all trained models (default + optimized)
for model_name, model_dict in trained_models.items():
    # DEFAULT
    if model_dict["default"] is not None:
        filename_default = models_output_path + f"{model_name}_DEFAULT_{rev_to_use}.sav"
        dump(model_dict["default"], open(filename_default, "wb"))

    # OPTIMIZED
    if model_dict["optimized"] is not None:
        filename_optimized = models_output_path + f"{model_name}_OPTIMIZED_{rev_to_use}.sav"
        dump(model_dict["optimized"], open(filename_optimized, "wb"))

print("\n✅ All models saved successfully.")